Egune Task for finetuning TTS models using Mongolian Dataset
> Add blockquote



In [1]:
!sudo apt-get install python3.9 python3.9-distutils python3.9-dev -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.9

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  python3.9-lib2to3 python3.9-minimal
Suggested packages:
  python3.9-venv binfmt-support
The following NEW packages will be installed:
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  python3.9 python3.9-dev python3.9-distutils python3.9-lib2to3
  python3.9-minimal
0 upgraded, 9 newly installed, 0 to remove and 38 not upgraded.
Need to get 12.2 MB of archives.
After this operation, 46.6 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.9-minimal amd64 3.9.24-1+jammy1 [838 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.9-minimal amd64 3.9.24-1+jammy1 [2,075 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubunt

In [2]:
!python3.9 -m pip install --upgrade pip setuptools wheel

In [3]:
#Check runtime on colab jupter notebook
!nvidia-smi

Thu Oct 16 15:30:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
#Pull github
!git clone https://github.com/dgduksict/tts-egune.git
%cd tts-egune

Cloning into 'tts-egune'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 45 (delta 20), reused 21 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 32.40 KiB | 4.05 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/tts-egune


In [5]:
#Download needed packages
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 180.5 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 155.2 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 174.8 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 71.8 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.7 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 193.3 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. T

In [6]:
#Import necessary libs
from huggingface_hub import notebook_login
from huggingface_hub import hf_hub_download
from datasets import load_dataset, Audio
import tarfile, os, pandas as pd

In [7]:
#Hugging face login

notebook_login()

In [8]:
#Define necessary functions
def example():
  return "hello duke"

example()

'hello duke'

In [9]:
from datasets import get_dataset_config_names
configs = get_dataset_config_names("mozilla-foundation/common_voice_17_0")
print(configs)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

common_voice_17_0.py:   0%|          | 0.00/8.19k [00:00<?, ?B/s]

RuntimeError: Dataset scripts are no longer supported, but found common_voice_17_0.py

In [47]:
dataset = load_dataset("mozilla-foundation/common_voice_17_0", "mn", split="train")
save_dir = "/commonvoice_mn"
os.makedirs(save_dir, exist_ok=True)

# Extract all tar contents (audio clips + TSV)
# for i, sample in enumerate(dataset):
#     file_path = sample["__key__"]
#     url = sample["__url__"]
#     if url.endswith(".tar") or url.endswith(".tar.gz"):
#         archive_path = os.path.join(save_dir, os.path.basename(url))
#         dataset.download_and_prepare()
#         with tarfile.open(archive_path, "r:*") as tar:
#             tar.extractall(save_dir)
#         break

# print("Extracted dataset to:", save_dir)
# !ls {save_dir}

# print(dataset.features)
print("Len: ", len(dataset))

RuntimeError: Dataset scripts are no longer supported, but found common_voice_17_0.py

In [26]:
with open(os.path.join(save_dir, "train.tsv"), "wb") as f:
    f.write(dataset[0]["tsv"])

print("✅ Saved train.tsv to:", save_dir)

✅ Saved train.tsv to: /commonvoice_mn


In [29]:
!head -n 10 /commonvoice_mn/train.tsv

clip	duration[ms]
common_voice_mn_40426968.mp3	2808
common_voice_mn_40262612.mp3	6084
common_voice_mn_40496857.mp3	4140
common_voice_mn_40508462.mp3	2808
common_voice_mn_40372331.mp3	3168
common_voice_mn_40471730.mp3	3420
common_voice_mn_40514106.mp3	3096
common_voice_mn_40460898.mp3	4752
common_voice_mn_20401673.mp3	7128


In [32]:
!ls /commonvoice_mn

train.tsv


In [33]:
df = pd.read_csv("/commonvoice_mn/train.tsv", sep="\t")
print(df.columns)
df.head()

Index(['clip', 'duration[ms]'], dtype='object')


,clip,duration[ms]
0,common_voice_mn_40426968.mp3,2808
1,common_voice_mn_40262612.mp3,6084
2,common_voice_mn_40496857.mp3,4140
3,common_voice_mn_40508462.mp3,2808
4,common_voice_mn_40372331.mp3,3168


In [18]:
# Calculate the number of rows for half the dataset
half_size = len(dataset) // 16

# Select the first half of the dataset
dataset = dataset.select(range(half_size))

print(dataset)

Dataset({
    features: ['tsv', '__key__', '__url__'],
    num_rows: 6018
})


In [ ]:
!ls